In [1]:
#[5] Certidão Negativa Justiça Federal:

In [2]:
#Bibliotecas:

#Deprecated:
#from selenium.webdriver import Chrome

#New Version: Selenium 4.0 Beta 1
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
 
#Options:
from selenium.webdriver.chrome.options import Options

#chrome_options.add_argument("--disable-extensions")
#chrome_options.add_argument('-headless')

from selenium.webdriver.support.ui import WebDriverWait

#Exceptions:
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException

from random import randrange       #random number (used in secs to wait - time)
import time                        #wait to run
import getpass                     #usr info
import pandas as pd
import json
import os

In [3]:
#Configurações Impressão em .pdf:
caminho_download = 'C:\\Users\\felip\\Downloads'
chrome_options = Options()
settings = {
       "recentDestinations": [{
            "id": "Save as PDF",
            "origin": "local",
            "account": "",
        }],
        "selectedDestinationId": "Save as PDF",
        "version": 2
    }
exclude_default_swithes = ["disable-client-side-phishing-detection",
    "safebrowsing-disable-auto-update",
    "safebrowsing-disable-download-protection",
    "ignore-certificate-errors",]
prefs = {"plugins.always_open_pdf_externally": True,
        "download.default_directory": caminho_download,       
        "printing.print_preview_sticky_settings.appState": json.dumps(settings),
        "savefile.default_directory": 'C:\\Users\\felip\\Downloads'}     #ADD
chrome_options.add_experimental_option("prefs",prefs)
chrome_options.add_argument('--kiosk-printing') #ADD
chrome_options.add_experimental_option("excludeSwitches", exclude_default_swithes)
chrome_options.add_experimental_option("useAutomationExtension", False)

In [4]:
#Cria uma pasta para a SC caso ela já não exista no sistema:
def criaPastaSC(codigo_SC): #Captura o código da SC
    newpath = r'C:\\Users\\felip\\OneDrive\\Documentos\\MEGA\\Documentacao\\Certidoes' 
    newpath = newpath + '\\' + codigo_SC
    if not os.path.exists(newpath):
        os.makedirs(newpath)

In [5]:
#Captura o nome do usuário:
usr = getpass.getuser()

In [6]:
#INPUTS:
df_cpf = pd.read_excel("C:/Users/felip/OneDrive/Documentos/MEGA/Documentacao/Robos/INPUT/A_Consultar.xlsx", engine='openpyxl')

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/felip/OneDrive/Documentos/MEGA/Documentacao/Robos/INPUT/A_Consultar.xlsx'

In [ ]:
#Acesso ao Site:

#Link para Certidão Negativa Justiça Federal:
url = "http://web.trf3.jus.br/certidao/Certidao/Solicitar"

#Varre cada CLiente/Pedido do Input:
for index, row in df_cpf.iterrows():
        
        #Identifica se a certidão conseguiu ser extraída atribuindo True:
        sucesso = False
        
        while sucesso != True:
        
            print("Realizando SC" + str(index))
        
            #Dados do Cliente:
            cpf = str(df_cpf['CPF'][index])
                #Tratamento da informação: '0    41810267870\nName: CPF, dtype: int64' para formato padrão.
            cpf2 = cpf.replace("0    ", "").replace("\nName: CPF, dtype: int64", "")

                #Tratamento da informação: '0    Felipe Borges dos Santos\nName: Nome, dtype: object' para formato padrão.
            nm = str(df_cpf['Nome'][index])
            nm2 = nm.replace("0    ", "").replace("\nName: Nome, dtype: object", "")
        
            #Inicializa o Chromedriver:
            #s = Service('chromedriver.exe')
            driver = webdriver.Chrome("chromedriver.exe", options=chrome_options)
                
            #Página Alvo:
            driver.get(url)

            #Marca a opção "Certidão de Distribuição" em Tipo de Certidão:
            time.sleep(2)
            elemento = (By.ID, "TipoCertidao4")
            acesso = driver.find_element(*elemento).click()

            #Marca a opção "Justiça Federal de Primeiro Grau em São Paulo" em Abrangência da certidão:
            time.sleep(2)
            elemento = (By.ID, "abrangenciaSJSP")
            acesso = driver.find_element(*elemento).click()

            #Preenche o nome completo:
            elemento = (By.ID, "Nome")
            acesso = driver.find_element(*elemento).send_keys(nm2)
            print(nm2)

            #marca  aopção de tipo de documento como "CPF":
            elemento = (By.ID, "PessoaFisicaComCpf")
            acesso = driver.find_element(*elemento).click()

            #Preenche com o número do CPF:
            elemento = (By.ID, "CpfCnpj")
            acesso = driver.find_element(*elemento).click()
            acesso = driver.find_element(*elemento).send_keys(cpf2)

            print(cpf2)

            #Espera um tempo aleatório entre 3 e 6 s para clicar em gerar a certidão:
            #time.sleep(randrange(4,8))
            time.sleep(randrange(6,8))

            print("esperando " + str(randrange(6,8)) + " para clicar")

            print("Clicando em Solicitar Certidão...")
            #Clica no Botão "Solicitar Certidão"

            try:
                element = WebDriverWait(driver, 4).until(EC.presence_of_element_located((By.ID, "BtGeraCerticao")))
                element.click()
            #Botão "Solicitar Certidão não encontrado"
            except NoSuchElementException:
                print("Botão de Solicitar Certidão não encontrado.")
            except TimeoutException:
                print("Botão - Solicitar Certificado - Demorou mais que 10 segundos ...")
                continue

            #Aguardando pelo botão de imprimir por 10 segundos... 
            #Espera explícita (baseada na visibilidade de dado elemento, se aparecer antes do timeout, o selenium prossegue. Se não > TimeoutException)
            #Espera implícita: funciona para qualquer comando que esteja presente no webdriver.
            print("Aguardando pelo botão imprimir...")

            #driver.implicitly_wait(10)

            #element = WebDriverWait(driver, 10).until(
            #EC.presence_of_element_located((By.ID, "botaoImprimir")))

            #Colocar um timer e clicar para gerar .pdf salvando em local apropriado:
            #ADD ID E EXPORTAR TB_TRATADOS > infos: RACF, HORARIO ENVIO

            try:

            #element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "botaoImprimir")))
                element = WebDriverWait(driver, 45000).until(EC.presence_of_element_located((By.ID, "botaoImprimir")))
                #element.click()
                
                print("Imprimindo...")
                
                #Cria Pasta na rede para a SC:
                criaPastaSC(index)
                
                #Imprime a tela através do CTRL+P:
                driver.execute_script('window.print();')
                sucesso = True
                
            except TimeoutException:
                print("ID:" + str(index))
                print("Demorou mais que 45 segundos...")
                continue

            except NoSuchElementException:
                print("ID:" + str(index))
                print("Botão de Solicitar Certidão não encontrado.")
                continue      